In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from astropy.io import fits
import pickle
from datetime import datetime
from scipy import signal

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")

def load_offsets(offset_file):
    """Load instrument offsets from CSV file"""
    if not os.path.exists(offset_file):
        print(f"Warning: Offset file {offset_file} not found. Using zero offsets.")
        return {}
    
    offsets = {}
    try:
        with open(offset_file, 'r') as f:
            lines = f.readlines()
            for line in lines[1:]:  # Skip header
                parts = line.strip().split(',')
                if len(parts) >= 2:
                    dataset = parts[0]
                    instrument_offsets = {}
                    # Parse instrument offsets (assuming format: DS1,harps:0.5,harpsn:1.2,etc.)
                    for i in range(1, len(parts)):
                        if ':' in parts[i]:
                            inst, offset = parts[i].split(':')
                            instrument_offsets[inst.lower()] = float(offset)
                    offsets[dataset] = instrument_offsets
    except Exception as e:
        print(f"Error reading offset file: {e}")
        return {}
    
    return offsets

def get_available_datasets(essp_dir):
    """Get list of available datasets"""
    datasets = []
    for item in os.listdir(essp_dir):
        if item.startswith('DS') and os.path.isdir(os.path.join(essp_dir, item)):
            try:
                ds_num = int(item[2:])  # Extract number from DS##
                datasets.append(ds_num)
            except ValueError:
                continue
    return sorted(datasets)

def analyze_ccfs_with_fiesta(essp_dir, dset_num, k_max=5, save_results=False, output_dir=None):
    """Apply FIESTA analysis to CCFs from a dataset using actual ESSP CCF errors"""
    try:
        from FIESTA_II import FIESTA  # Import here to avoid issues if not available
    except ImportError:
        print("Error: FIESTA_II module not found. Please install FIESTA first.")
        return None
    
    print(f"\n=== FIESTA Analysis for DS{dset_num} ===")
    
    instruments = ['harpsn', 'harps', 'expres', 'neid']
    ccf_dir = os.path.join(essp_dir, f'DS{dset_num}', 'CCFs')
    
    if not os.path.exists(ccf_dir):
        print(f"Warning: {ccf_dir} not found")
        return None
    
    results = {}
    
    for inst in instruments:
        print(f"\nProcessing {inst.upper()}...")
        
        # Get all files for this instrument
        files = sorted(glob(os.path.join(ccf_dir, f'DS{dset_num}.*_ccfs_{inst}.fits')))
        if not files:
            print(f"  No files found for {inst}")
            continue
        
        # Process files to extract CCF data with proper errors
        ccf_data = []
        ccf_errors = []
        v_grids = []
        emjd_times = []
        file_names = []
        
        for file in files:
            try:
                hdus = fits.open(file)
                
                # Extract data from FITS file (using CORRECT UPPERCASE extension names from documentation)
                v_grid = hdus['V_GRID'].data.copy()  # Velocity grid [km/s]
                ccf = hdus['CCF'].data.copy()        # Global CCF
                e_ccf = hdus['E_CCF'].data.copy()    # Global CCF errors
                
                # Extract eMJD from FITS header TIME field (keep as eMJD)
                try:
                    if 'TIME' in hdus[0].header:
                        emjd = hdus[0].header['TIME']  # Extended Modified Julian Date
                        emjd_times.append(emjd)
                    else:
                        print(f"    Warning: No TIME in header for {os.path.basename(file)}, using index")
                        emjd_times.append(len(emjd_times))  # Use index as fallback
                        
                except Exception as e:
                    print(f"    Warning: Could not extract eMJD from {os.path.basename(file)}: {e}")
                    emjd_times.append(len(emjd_times))  # Use index as fallback
                
                hdus.close()
                
                # Check for valid data
                if np.sum(np.isfinite(ccf)) == 0:
                    print(f"    Skipping {os.path.basename(file)}: no valid CCF data")
                    continue
                
                # Check for valid errors
                if np.sum(np.isfinite(e_ccf)) == 0:
                    print(f"    Warning: {os.path.basename(file)}: no valid error data, estimating errors")
                    # Fallback error estimation if E_CCF is invalid
                    ccf_max = np.nanmax(ccf)
                    ccf_depth = ccf_max - np.nanmin(ccf)
                    e_ccf = np.sqrt(np.abs(ccf)) * 0.01 + ccf_depth * 0.001
                
                ccf_data.append(ccf)
                ccf_errors.append(e_ccf)
                v_grids.append(v_grid)
                file_names.append(os.path.basename(file))
                
            except KeyError as e:
                print(f"    Error: Missing FITS extension in {os.path.basename(file)}: {e}")
                print(f"    Available extensions: {[hdu.name for hdu in hdus]}")
                hdus.close()
                continue
            except Exception as e:
                print(f"    Error processing {os.path.basename(file)}: {e}")
                continue
        
        if len(ccf_data) < 2:
            print(f"  Insufficient data for {inst} (need at least 2 CCFs, got {len(ccf_data)})")
            continue
        
        # Convert to numpy arrays
        n_files = len(ccf_data)
        n_points = len(v_grids[0])
        
        # Create CCF matrix: [n_velocity_points, n_observations]
        CCF = np.zeros((n_points, n_files))
        eCCF = np.zeros((n_points, n_files))
        V_grid = v_grids[0]  # Assuming all have same velocity grid
        
        # Normalize CCFs and handle errors properly
        for i, (ccf, e_ccf) in enumerate(zip(ccf_data, ccf_errors)):
            # Find normalization factor
            ccf_max = np.nanmax(ccf)
            if ccf_max <= 0:
                print(f"    Skipping observation {i}: invalid CCF maximum")
                continue
            
            # Normalize CCF
            ccf_norm = ccf / ccf_max
            CCF[:, i] = ccf_norm
            
            # Propagate errors through normalization: e_norm = e_ccf / ccf_max
            e_ccf_norm = e_ccf / ccf_max
            
            # Handle any remaining invalid errors
            invalid_errors = ~np.isfinite(e_ccf_norm) | (e_ccf_norm <= 0)
            if np.any(invalid_errors):
                # Use photon noise estimate for invalid errors
                e_ccf_norm[invalid_errors] = np.sqrt(np.abs(ccf_norm[invalid_errors])) * 0.01
            
            eCCF[:, i] = e_ccf_norm
        
        # Apply FIESTA
        try:
            print(f"  Applying FIESTA with k_max={k_max} to {n_files} CCFs...")
            
            # FIESTA returns 6 values when noise is present:
            # df, v_k, σv_k, A_k, σA_k, RV_gauss
            result = FIESTA(
                V_grid, CCF, eCCF, 
                template=[],  # Let FIESTA calculate template automatically
                SNR=2.0,
                k_max=k_max
            )
            
            if len(result) == 6:
                # With noise (normal case)
                df, v_k, sigma_v_k, A_k, sigma_A_k, RV_gauss = result
                
                # Check if RV_gauss is flat (all same values)
                if np.std(RV_gauss) < 1e-6:  # Very small variation
                    print(f"  Warning: RV_gauss appears flat (std={np.std(RV_gauss):.2e})")
                    print(f"  Using first FIESTA mode (k=1) as reference instead")
                    # Use first mode as reference for differential RVs
                    RV_reference = RV_gauss * 1000  # Convert to m/s
                
                # v_k contains the RV variations for each mode
                RV_FT_k = v_k * 1000  # Convert to m/s
                eRV_FT_k = sigma_v_k * 1000  # Convert uncertainties to m/s
                
            elif len(result) == 3:
                # Without noise (unlikely)
                v_k, A_k, RV_gauss = result
                RV_FT_k = v_k * 1000
                eRV_FT_k = np.ones_like(v_k) * 100  # Default errors in m/s
                RV_reference = RV_gauss * 1000
                
            else:
                print(f"  Warning: FIESTA returned unexpected {len(result)} values")
                continue
            
            # Calculate differential RVs (activity indicators)
            ΔRV_k = np.zeros(RV_FT_k.shape)
            for k in range(RV_FT_k.shape[0]):
                ΔRV_k[k, :] = RV_FT_k[k, :] - RV_reference
            
            # Debug print to check what's happening
            print(f"  Debug: RV_gauss range: {np.min(RV_reference):.2f} to {np.max(RV_reference):.2f} m/s")
            print(f"  Debug: Mode 1 range: {np.min(RV_FT_k[0, :]):.2f} to {np.max(RV_FT_k[0, :]):.2f} m/s")
            print(f"  Debug: Are RV_gauss and Mode 1 the same? {np.allclose(RV_reference, RV_FT_k[0, :])}")
            
            # Store results
            results[inst] = {
                'files': file_names,
                'emjd_times': np.array(emjd_times[:n_files]),  # ← REAL eMJD TIMES
                'V_grid': V_grid,
                'CCF': CCF,
                'eCCF': eCCF,
                'RV_FT_k': RV_FT_k,      # RV from each Fourier mode [m/s]
                'eRV_FT_k': eRV_FT_k,    # Uncertainties on RV_FT_k [m/s]
                'A_k': A_k,              # Amplitude of each Fourier mode
                'RV_gauss': RV_reference,  # Reference RV [m/s]
                'ΔRV_k': ΔRV_k,          # Differential RVs (activity indicators) [m/s]
                'n_files': n_files,
                'k_max': RV_FT_k.shape[0]  # Actual number of modes returned
            }
            
            print(f"  Success! Processed {n_files} CCFs with {RV_FT_k.shape[0]} FIESTA modes")
            print(f"  RV range: {np.min(RV_reference):.1f} to {np.max(RV_reference):.1f} m/s")
            print(f"  eMJD range: {np.min(emjd_times):.6f} to {np.max(emjd_times):.6f}")  # Changed from BJD
            print(f"  Activity indicator ranges:")
            for k in range(min(3, RV_FT_k.shape[0])):
                rms = np.std(ΔRV_k[k, :])
                print(f"    ΔRV_{k+1}: RMS = {rms:.2f} m/s")
            
        except Exception as e:
            print(f"  Error applying FIESTA to {inst}: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    return results

def periodogram6(ax, time, data, vlines=None):
    """Calculate and plot periodogram similar to the reference code"""
    from scipy import signal
    
    # Remove mean
    data_centered = data - np.mean(data)
    
    # Calculate Lomb-Scargle periodogram
    frequencies = np.logspace(-2, 2, 1000)  # From 0.01 to 100 days
    periods = 1.0 / frequencies
    
    try:
        power = signal.lombscargle(time, data_centered, frequencies, normalize=True)
        
        # Plot periodogram
        ax.semilogx(periods, power, 'k-', alpha=0.7, linewidth=1)
        
        # Add vertical lines for specific periods
        if vlines:
            for period in vlines:
                ax.axvline(period, color='red', linestyle='--', alpha=0.7, linewidth=1)
                ax.text(period, ax.get_ylim()[1]*0.9, f'{period}', 
                       rotation=90, ha='right', va='top', fontsize=8, color='red')
        
        ax.set_xlim(1, 100)
        ax.set_ylabel('Power')
        
    except Exception as e:
        print(f"Error calculating periodogram: {e}")
        ax.text(0.5, 0.5, 'Periodogram\nError', ha='center', va='center', 
                transform=ax.transAxes)

def plot_fiesta_results(results, dset_num, save_plots=True, output_dir=None):
    """Plot FIESTA results in the same style as the reference figure"""
    if not results:
        print("No results to plot")
        return
    
    print(f"\nCreating FIESTA-style plots for DS{dset_num}...")
    
    # Combine all instruments data
    all_times = []
    all_rv_gauss = []
    all_delta_rvs = []
    
    for inst in results:
        data = results[inst]
        times = data['emjd_times'] - np.min(data['emjd_times'])  # Convert to days from start (eMJD)
        rv_gauss = data['RV_gauss']
        delta_rvs = data['ΔRV_k']
        
        all_times.extend(times)
        all_rv_gauss.extend(rv_gauss)
        
        # Extend delta RVs
        if len(all_delta_rvs) == 0:
            all_delta_rvs = [[] for _ in range(delta_rvs.shape[0])]
        
        for k in range(delta_rvs.shape[0]):
            all_delta_rvs[k].extend(delta_rvs[k, :])
    
    # Convert to numpy arrays
    all_times = np.array(all_times)
    all_rv_gauss = np.array(all_rv_gauss)
    all_delta_rvs = [np.array(drv) for drv in all_delta_rvs]
    
    # Determine number of modes
    k_mode = len(all_delta_rvs)
    N_file = len(all_times)
    
    # Set up the plot layout (same as reference)
    alpha1 = 0.2
    plt.rcParams.update({'font.size': 10})
    lw = 2
    widths = [7, 1, 7]
    heights = [1] * (k_mode + 1)  # +1 for RV_gauss row
    gs_kw = dict(width_ratios=widths, height_ratios=heights)
    
    fig, f6_axes = plt.subplots(figsize=(10, k_mode + 1), ncols=3, nrows=k_mode + 1, 
                               constrained_layout=True, gridspec_kw=gs_kw)
    
    # Import LinearRegression for R² calculation
    try:
        from sklearn.linear_model import LinearRegression
    except ImportError:
        print("Warning: sklearn not available, R² calculation will be skipped")
        LinearRegression = None
    
    # Plot each row
    for r in range(k_mode + 1):
        for c in range(3):
            ax = f6_axes[r, c]
            
            # Column 0: Time-series
            if c == 0:
                if r == 0:
                    # First row: RV_gauss)
                    ax.plot(all_times, all_rv_gauss, 'k.', alpha=alpha1)
                    ax.set_title('Time-series')
                    ax.set_ylabel('$RV_{gauss}$')
                else:
                    # Other rows: ΔRV_k
                    ax.plot(all_times, all_delta_rvs[r-1], 'k.', alpha=alpha1)
                    ax.set_ylabel(r'$\Delta$RV$_{%d}$' % r)
                
                if r != k_mode:
                    ax.tick_params(labelbottom=False)
                else:
                    ax.set_xlabel('Time [d]')
            
            # Column 1: R² correlation plots
            elif c == 1:
                if LinearRegression is not None:
                    if r == 0:
                        # RV_gauss vs RV_gauss (perfect correlation)
                        reg = LinearRegression().fit(all_rv_gauss.reshape(-1, 1), 
                                                   all_rv_gauss.reshape(-1, 1))
                        score = reg.score(all_rv_gauss.reshape(-1, 1), 
                                        all_rv_gauss.reshape(-1, 1))
                        adjust_R2 = 1 - (1 - score) * (N_file - 1) / (N_file - 1 - 1)
                        title = r'$\bar{R}$' + r'$^2$'
                        ax.set_title(title + ' = {:.2f}'.format(adjust_R2))
                        ax.plot(all_rv_gauss, all_rv_gauss, 'k.', alpha=alpha1)
                    else:
                        # ΔRV_k vs RV_gauss correlation
                        reg = LinearRegression().fit(all_rv_gauss.reshape(-1, 1), 
                                                   all_delta_rvs[r-1].reshape(-1, 1))
                        score = reg.score(all_rv_gauss.reshape(-1, 1), 
                                        all_delta_rvs[r-1].reshape(-1, 1))
                        adjust_R2 = 1 - (1 - score) * (N_file - 1) / (N_file - 1 - 1)
                        title = r'$\bar{R}$' + r'$^2$'
                        ax.set_title(title + ' = {:.2f}'.format(adjust_R2))
                        ax.plot(all_rv_gauss, all_delta_rvs[r-1], 'k.', alpha=alpha1)
                else:
                    ax.text(0.5, 0.5, 'R² calc\nunavailable', ha='center', va='center',
                           transform=ax.transAxes)
                
                if r != k_mode:
                    ax.tick_params(labelbottom=False)
                else:
                    ax.set_xlabel('$RV_{gauss}$')
                ax.yaxis.tick_right()
            
            # Column 2: Periodograms
            elif c == 2:
                if r == 0:
                    # RV_gauss periodogram
                    periodogram6(ax, all_times, all_rv_gauss, vlines=[8.0, 14.6, 26.6])
                    ax.set_title('Periodogram')
                else:
                    # ΔRV_k periodogram
                    periodogram6(ax, all_times, all_delta_rvs[r-1], vlines=[8.0, 14.6, 26.6])
                
                if r != k_mode:
                    ax.tick_params(labelbottom=False)
                if r == k_mode:
                    ax.set_xlabel('Period [days]')
    
    # Align y-labels
    fig.align_ylabels(f6_axes[:, 0])
    
    # Save or show
    if save_plots and output_dir:
        filename = os.path.join(output_dir, f'DS{dset_num}_FIESTA_style.png')
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved: {filename}")
    else:
        plt.show()

def save_activity_indicators_dat(results, dset_num, output_dir):
    """Save FIESTA activity indicators (differential RVs) to .dat files with eMJD times"""
    if not results:
        print("No FIESTA results to save")
        return
    
    print(f"\nSaving activity indicators to .dat files with eMJD times...")
    
    # Set the correct output directory for .dat files
    dat_output_dir = '/work2/lbuc/iara/GitHub/PyORBIT_examples/ESSP4/data'
    os.makedirs(dat_output_dir, exist_ok=True)
    
    for inst in results:
        data = results[inst]
        n_obs = data['n_files']
        k_max = data['k_max']
        
        print(f"  Processing {inst.upper()}...")
        
        # Handle each instrument separately (including harps and harpsn)
        for k in range(k_max):
            filename = os.path.join(dat_output_dir, f'DS{dset_num}_{inst}_fiesta_mode{k+1}.dat')
            
            # Use eMJD times from FIESTA results
            emjd_times = data['emjd_times']                    # ← eMJD VALUES
            delta_rv = data['ΔRV_k'][k, :]                  # ← Differential RV for mode k
            
            # Get uncertainties
            if 'eRV_FT_k' in data:
                uncertainties = data['eRV_FT_k'][k, :]
            else:
                # Estimate from RMS if formal errors not available
                uncertainties = np.full(n_obs, np.std(delta_rv))
            
            # Create other columns (same format as Contrast.dat)
            jitter = np.zeros(n_obs)
            offset = np.zeros(n_obs)  # Single instrument = single offset
            flag = np.full(n_obs, -1, dtype=int)  # ← INTEGER FLAGS
            
            # Stack data: eMJD, value, error, jitter, offset, flag
            output_data = np.column_stack([
                emjd_times,     # ← eMJD VALUES
                delta_rv,       # ← FIESTA differential RV
                uncertainties,  # ← Uncertainties
                jitter,         # ← Jitter (0)
                offset,         # ← Offset (0)
                flag            # ← Flag (-1 as integer)
            ])
            
            # Save with same format as Contrast.dat - FLAGS AS INTEGERS
            np.savetxt(filename, output_data, 
                     fmt=['%.6f', '%.6f', '%.6f', '%d', '%d', '%d'])
            
            print(f"    Saved: {filename} (n_obs={n_obs})")

    print(f"All FIESTA activity indicator files saved with eMJD times!")




: 

In [ ]:
def main():
    """Main function to run FIESTA analysis"""
    # ========== EDIT THESE SETTINGS ==========
    essp_dir = '/work2/lbuc/data/ESSP4/ESSP4'
    output_dir = '/work2/lbuc/iara/GitHub/ESSP/Figures/FIESTA_figures'
    offset_filename = 'instrument_offsets_iccf.csv'
    
    # FIESTA settings
    dataset = None  # Specific dataset number, or None for all datasets
    k_max = 5      # Number of FIESTA modes
    save_results = False  # No pickle files needed
    save_plots = True
    # =========================================
    
    print(f"FIESTA Analysis for ESSP4 CCF Data")
    print(f"Data directory: {essp_dir}")
    print(f"Output directory for plots: {output_dir}")
    print(f"Output directory for .dat files: /work2/lbuc/iara/GitHub/PyORBIT_examples/ESSP4/data")
    print(f"FIESTA modes: {k_max}")
    print(f"Save plots: {save_plots}")
    
    if not os.path.exists(essp_dir):
        print(f"Error: {essp_dir} not found")
        return
    
    # Create output directory for plots
    if save_plots:
        os.makedirs(output_dir, exist_ok=True)
    
    # Load offsets (optional, for future use)
    offset_file = os.path.join(essp_dir, offset_filename)
    offset_dict = load_offsets(offset_file)
    
    # Get datasets
    if dataset:
        datasets = [dataset]
    else:
        datasets = get_available_datasets(essp_dir)
    
    print(f"Analyzing datasets: {datasets}")
    
    for dset_num in datasets:
        print(f"\n{'='*60}")
        print(f"Processing Dataset {dset_num}")
        print(f"{'='*60}")
        
        # Run FIESTA analysis
        fiesta_results = analyze_ccfs_with_fiesta(
            essp_dir, dset_num, k_max=k_max,
            save_results=save_results,
            output_dir=output_dir if save_results else None
        )
        
        if fiesta_results:
            # Save activity indicators to .dat files
            save_activity_indicators_dat(
                fiesta_results, dset_num, output_dir
            )
            
            # Plot main results (only differential RVs)
            plot_fiesta_results(
                fiesta_results, dset_num,
                save_plots=save_plots,
                output_dir=output_dir if save_plots else None
            )
        
        print(f"Completed analysis for DS{dset_num}")
    
    print(f"\n{'='*60}")
    print("FIESTA Analysis Complete!")
    print(f"Plots saved to: {output_dir}")
    print(f".dat files saved to: /work2/lbuc/iara/GitHub/PyORBIT_examples/ESSP4/data")
    print(f"{'='*60}")

if __name__ == "__main__":
    main()

FIESTA Analysis for ESSP4 CCF Data
Data directory: /work2/lbuc/data/ESSP4/ESSP4
Output directory for plots: /work2/lbuc/iara/GitHub/ESSP/Figures/FIESTA_figures
Output directory for .dat files: /work2/lbuc/iara/GitHub/PyORBIT_examples/ESSP4/data
FIESTA modes: 5
Save plots: True
Analyzing datasets: [1, 2, 3, 4, 5, 6, 7, 8, 9]

Processing Dataset 1

=== FIESTA Analysis for DS1 ===

Processing HARPSN...
  Applying FIESTA with k_max=5 to 84 CCFs...


/work2/lbuc/jzhao/conda_envs/pyorbit/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',



Velocity grid used [-12.00, 12.00]
The median SNR of all CCFs is 1573
The user-defined SNR = 2.0

ξ_photon = 0.70
ξ_normal = 1.23
ξ_individual = 1.23
ξ_timeseries = 1.23

In summary, the cut-off frequency is recommended to be 0.70 (k_max = 17)

        ξ  individual_SNR  ts_SNR_A  ts_SNR_ϕ  modelling noise  \
0   0.000        240227.7      54.7       0.0          0.17799   
1   0.041         32872.3      13.9      26.3          0.07236   
2   0.082         14213.8      15.0      18.6          0.01719   
3   0.123          3384.1      10.6      11.2          0.00397   
4   0.164           760.7       8.4       9.3          0.00132   
5   0.205            36.2       8.9      11.3          0.00130   
6   0.246           119.2       9.3       9.1          0.00116   
7   0.287            94.0       9.2       8.8          0.00107   
8   0.328            78.3       7.7       7.6          0.00099   
9   0.369            73.2       8.0       8.0          0.00093   
10  0.410            62.6   

/work2/lbuc/jzhao/conda_envs/pyorbit/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


The median SNR of all CCFs is 1470
The user-defined SNR = 2.0

ξ_photon = 0.78
ξ_normal = 1.23
ξ_individual = 1.23
ξ_timeseries = 1.02

In summary, the cut-off frequency is recommended to be 0.78 (k_max = 19)

        ξ  individual_SNR  ts_SNR_A  ts_SNR_ϕ  modelling noise  \
0   0.000        212557.5      68.3       0.0          0.17053   
1   0.041         27914.0      13.2      18.6          0.06802   
2   0.082         11807.3       9.9      16.9          0.01591   
3   0.123          2761.6       9.8      10.0          0.00383   
4   0.164           620.1      10.9       9.5          0.00165   
5   0.205            78.6       7.3       8.9          0.00159   
6   0.246           125.2       8.1       7.2          0.00143   
7   0.287            97.7       8.6       6.4          0.00132   
8   0.328            89.9       5.8       4.7          0.00122   
9   0.369            79.4       8.8       6.8          0.00115   
10  0.410            66.6       8.6       5.6          0.00108  

...

/work2/lbuc/jzhao/conda_envs/pyorbit/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


The median SNR of all CCFs is 1165
The user-defined SNR = 2.0

ξ_photon = 0.90
ξ_normal = 1.23
ξ_individual = 1.23
ξ_timeseries = 1.19

In summary, the cut-off frequency is recommended to be 0.90 (k_max = 22)

        ξ  individual_SNR  ts_SNR_A  ts_SNR_ϕ  modelling noise  \
0   0.000        182351.3      55.9       0.0          0.16084   
1   0.041         22232.3      12.2      11.3          0.06662   
2   0.082          9697.3      20.8      10.3          0.01894   
3   0.123          2772.9      12.4       7.2          0.00478   
4   0.164           597.9       7.9       6.8          0.00268   
5   0.205           171.2       7.0       6.2          0.00242   
6   0.246           168.5       6.9       8.4          0.00215   
7   0.287           118.7       6.3       6.3          0.00199   
8   0.328           112.2       6.7       6.4          0.00185   
9   0.369            96.4       7.3       7.1          0.00174   
10  0.410            88.9       5.6       6.2          0.00164  

/work2/lbuc/jzhao/conda_envs/pyorbit/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:1010: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


The median SNR of all CCFs is 1128
The user-defined SNR = 2.0

ξ_photon = 0.61
ξ_normal = 1.23
ξ_individual = 1.23
ξ_timeseries = 1.11

In summary, the cut-off frequency is recommended to be 0.61 (k_max = 15)

        ξ  individual_SNR  ts_SNR_A  ts_SNR_ϕ  modelling noise  \
0   0.000        172698.5      33.0       0.0          0.15979   
1   0.041         21058.9      11.3       8.5          0.06311   
2   0.082          8732.7      12.5       8.3          0.01667   
3   0.123          2329.7       7.0       6.8          0.00383   
4   0.164           487.5       5.2       5.1          0.00179   
5   0.205           104.7       4.6       3.8          0.00164   
6   0.246           105.5       4.0       4.5          0.00147   
7   0.287            77.4       3.7       3.2          0.00136   
8   0.328            72.2       3.3       3.5          0.00126   
9   0.369            63.0       3.1       2.9          0.00119   
10  0.410            57.8       3.2       3.0          0.00112  

/work2/lbuc/jzhao/conda_envs/pyorbit/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:152: RuntimeWarning: divide by zero encountered in scalar divide
  pgram *= 2 / np.dot(y, y)
/work2/lbuc/jzhao/conda_envs/pyorbit/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:152: RuntimeWarning: invalid value encountered in multiply
  pgram *= 2 / np.dot(y, y)
